# Imports und Sedona-Kontext (sed)

In [2]:
from sedona.spark import *
from pyspark.sql.functions import expr
import geopandas as gpd


_config = SedonaContext.builder().master("spark://localhost:7077") .\
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all'). \
    getOrCreate()

sed = SedonaContext.create(_config)

https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/25 19:01:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Gemeinden (gem)

In [8]:
_p_bound = '../data/ch/boundaries/swissBOUNDARIES3D_1_5_TLM_HOHEITSGEBIET.shp'

_pdf_gem = gpd.read_file(_p_bound)
gem = sed.createDataFrame(_pdf_gem)
gem = gem.select(gem.OBJEKTART.alias('art'), gem.KANTONSNUM.alias('kant'), gem.BFS_NUMMER.alias('bfs'), gem.NAME.alias('name'), gem.EINWOHNERZ.alias('pers'), gem.geometry)
gem = gem.filter(gem.kant.eqNullSafe(5))

gem.alias('gem')
gem.show(2)


24/02/25 19:05:14 WARN TaskSetManager: Stage 6 contains a task of very large size (13567 KiB). The maximum recommended task size is 1000 KiB.


+--------------+----+----+----------+-----+--------------------+
|           art|kant| bfs|      name| pers|            geometry|
+--------------+----+----+----------+-----+--------------------+
|Gemeindegebiet| 5.0|1367| Muotathal| 3516|POLYGON ((2707843...|
|Gemeindegebiet| 5.0|1301|Einsiedeln|16310|POLYGON ((2701109...|
+--------------+----+----+----------+-----+--------------------+
only showing top 2 rows



# Wohngebiete pro Gemeinde (wgg)

Die TLM Siedlungsgebiete werden an den Gemeindegrenzen verschnitten. Die resultierende "Quartiere" der Gemeinde zugeordnet, in welcher ihr Zentrumspunkt zu liegen kommt.

In [25]:
_p_cover = '../data/ch/tlm_reg/swisstlmregio_product_lv95/Landcover/swissTLMRegio_LandCover.shp'

_pdf_cover = gpd.read_file(_p_cover)
wg = sed.createDataFrame(_pdf_cover)
wg = wg.select(wg.OBJVAL.alias('art'), wg.geometry.alias('poly'))
wg = wg.filter('art in ("Siedl","Stadtzentr")')

gem.createOrReplaceTempView('gem')
wg.createOrReplaceTempView('wg')

wgg = sed.sql('select ST_Intersection(wg.poly, gem.geometry) as poly from gem, wg')
wgg.createOrReplaceTempView('wgg')

wgg = sed.sql('select gem.bfs, gem.kant, gem.name, gem.pers, ST_Area(wgg.poly) as quart_fl, wgg.poly from wgg inner join gem on ST_Within(ST_Centroid(wgg.poly), gem.geometry)')
wgg.createOrReplaceTempView('wgg')

# Personen mittels Flächenverhältnis auf die Quartiere verteilen

Verteilt die Einwohner der Gemeinde auf die Quartiere gemäss dem Verhältnis der Quartierflächen zur Gesamtfläche aller Quartiere einer Gemeinde.

Könnte man wahrscheinlich noch "sparkiger" machen, aber so funktioniert es auch

In [30]:
fl_summ = sed.sql('select wgg.bfs, sum(ST_Area(poly)) as tot_fl from wgg group by bfs')
quart = wgg.join(fl_summ, wgg.bfs == fl_summ.bfs).withColumn('quart_pers', wgg.pers * wgg.quart_fl / fl_summ.tot_fl)

quart.show(10)

24/02/25 20:39:15 WARN TaskSetManager: Stage 54 contains a task of very large size (28004 KiB). The maximum recommended task size is 1000 KiB.
24/02/25 20:39:23 WARN TaskSetManager: Stage 55 contains a task of very large size (28004 KiB). The maximum recommended task size is 1000 KiB.
24/02/25 20:39:32 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.
24/02/25 20:39:32 WARN TaskSetManager: Stage 56 contains a task of very large size (13567 KiB). The maximum recommended task size is 1000 KiB.
24/02/25 20:39:33 WARN TaskSetManager: Stage 57 contains a task of very large size (28004 KiB). The maximum recommended task size is 1000 KiB.
24/02/25 20:39:41 WARN TaskSetManager: Stage 59 contains a task of very large size (28004 KiB). The maximum recommended task size is 1000 KiB.
24/02/25 20:39:49 WARN TaskSetManager: Stage 60 contains a task of very large size (28004 KiB). The maximum recommended task size is 1000 KiB.
24/02/25 20:39:58 WARN JoinQuery: UseInd

+----+----+-----------+-----+------------------+--------------------+----+------------------+------------------+
| bfs|kant|       name| pers|          quart_fl|                poly| bfs|            tot_fl|        quart_pers|
+----+----+-----------+-----+------------------+--------------------+----+------------------+------------------+
|1371| 5.0|     Sattel| 2021|          221929.0|POLYGON ((2691552...|1371|1132879.7963902224| 395.9100607400249|
|1371| 5.0|     Sattel| 2021| 910950.7963902225|POLYGON ((2691097...|1371|1132879.7963902224|1625.0899392599752|
|1374| 5.0|Steinerberg|  951|503351.39061434654|POLYGON ((2687459...|1374|503351.39061434654|             951.0|
|1343| 5.0|  Innerthal|  180|          163002.5|POLYGON ((2712574...|1343|          163002.5|             180.0|
|1362| 5.0|       Arth|12359| 47320.22826634711|POLYGON ((2678572...|1362|3366461.5815641894|173.72267200270522|
|1362| 5.0|       Arth|12359| 3319141.353297842|POLYGON ((2683157...|1362|3366461.5815641894|121